### Smile On EDA

#### For this analysis we want to answer the following questions: 
-  Has the Smile On program been effective?
-  Is the Smile On program being used by people in the area where tooth loss is greatest?

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

#### Let's read in the data and look at the first few rows

In [16]:
smile = pd.read_csv('data/smile_on.csv', sep=',', encoding='latin-1', skiprows = 2)
smile.head()

C:\Users\sbive\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32,36,53,59,88,89,127,141,144,156,157,158,159) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,81381827,SMILE ON 60+ Registration,NaN,11/25/2020 16:46,11/25/2020 16:48,2,3859874,136529,NaN,NaN,...,NaN,NaN,Matthew Walker Comprehensive Health Center Inc...,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN
1,81379177,SMILE ON 60+ Registration,NaN,11/25/2020 15:40,11/25/2020 15:55,15,3859755,82341,NaN,NaN,...,NaN,NaN,NaN,Health Assist Office,NaN,NaN,NaN,NaN,NaN,NaN
2,81377435,SMILE ON 60+ Clinical,NaN,11/25/2020 15:02,11/25/2020 15:02,0,2518680,119506,NaN,NaN,...,NaN,NaN,Friends in Need Health Center - Kingsport,Provider Site,NaN,Yes,Yes,Yes,NaN,NaN
3,81375215,SMILE ON 60+ Clinical,NaN,11/25/2020 14:17,11/25/2020 14:17,0,3231923,143703,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Frayser,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN
4,81373934,SMILE ON 60+ Clinical,NaN,11/25/2020 13:50,11/25/2020 13:58,8,-1,135611,NaN,NaN,...,Urgent Care - within next week-pain or infection,Yes,Neighborhood Health - Lebanon,NaN,NaN,NaN,Yes,No,NaN,Yes


In [15]:
smile.shape

(40032, 184)

In [17]:
smile.info

<bound method DataFrame.info of        CallReportNum              ReportVersion  LinkedToCallReportNum  \
0           81381827  SMILE ON 60+ Registration                    NaN   
1           81379177  SMILE ON 60+ Registration                    NaN   
2           81377435      SMILE ON 60+ Clinical                    NaN   
3           81375215      SMILE ON 60+ Clinical                    NaN   
4           81373934      SMILE ON 60+ Clinical                    NaN   
...              ...                        ...                    ...   
40027       53192452  SMILE ON 60+ Registration                    NaN   
40028       67915561  SMILE ON 60+ Registration                    NaN   
40029       67251405      SMILE ON 60+ Clinical                    NaN   
40030       63063150      SMILE ON 60+ Clinical                    NaN   
40031       70839903      SMILE ON 60+ Clinical                    NaN   

      CallDateAndTimeStart CallDateAndTimeEnd  CallLength  CallerNum  \
0      

In [19]:
smile.CallerNum.value_counts()

-1          70
 2441776    36
 2431132    31
 2416076    29
 2466994    28
            ..
 3825101     1
 3743280     1
 2653820     1
 3686036     1
 3821558     1
Name: CallerNum, Length: 8708, dtype: int64

In [38]:
smile.CityName.value_counts().head(10)

Nashville       6456
Memphis         6304
Murfreesboro    2435
Cleveland       1718
Knoxville       1461
Kingsport       1219
Antioch          976
Madison          733
Johnson City     704
Smyrna           650
Name: CityName, dtype: int64

In [37]:
smile['Birth Year'].value_counts().head(10)

1958    2182
1957    2046
1956    2036
1955    1841
1954    1670
1956    1493
1955    1444
1958    1364
1957    1354
1953    1286
Name: Birth Year, dtype: int64

In [36]:
smile['SMILE ON 60+ Site Information - Enroller ID'].value_counts().head(10)

Interfaith Dental Clinic - Nashville                           5802
Interfaith Dental Clinic - Murfreesboro                        3706
Church Health Center - Memphis                                 3581
Matthew Walker Comprehensive Health Center Inc. - Nashville    2516
Interfaith Health Clinic of Knoxville - Knoxville              1822
Karis Dental Clinic - Cleveland                                1809
Friends in Need Health Center - Kingsport                      1566
Healing Hands Health Center - Bristol                          1346
Keystone Dental Care Inc. - Johnson City                       1032
Christ Community Health Services- Hickory Hill                  986
Name: SMILE ON 60+ Site Information - Enroller ID, dtype: int64

In [39]:
smile['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?'].value_counts().head(10)

Yes    15930
No      1098
Name: SMILE ON 60+ Treatment Plan - Was a treatment plan developed?, dtype: int64

In [40]:
smile['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?'].value_counts().head(10)

No     12431
Yes     5104
Name: SMILE ON 60+ Treatment Plan - Is the treatment plan completed?, dtype: int64

In [41]:
smile['SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'].value_counts().head(10)

No     8127
Yes    6149
Name: SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free), dtype: int64

In [54]:
smile['SMILE ON 60+ Demographic Information - How did you first hear about the SMILE ON 60+ program?'].value_counts().head(10)

Dental Clinic Staff                  22805
Other                                 5725
This encounter                        2488
Wisdom Tooth Project Presentation     1988
SMILE ON CDHC                         1438
Senior Center                         1431
TV                                     983
Flyer                                  511
Newspaper                              258
Dental Hotline                         145
Name: SMILE ON 60+ Demographic Information - How did you first hear about the SMILE ON 60+ program?, dtype: int64

In [56]:
smile['SMILE ON 60+ Verification of Eligibility - Transportation Arranged'].value_counts().head(10)

No     5483
Yes     342
Name: SMILE ON 60+ Verification of Eligibility - Transportation Arranged, dtype: int64

In [57]:
smile['SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?'].value_counts().head(10)

Less than 12 months    16751
More than 5 years       8533
1 to 2 years            7077
3 to 5 years            4863
Name: SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?, dtype: int64